In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type

In [74]:
df = pd.read_csv('csv/megamillions.csv', names=['name','month','day','year','s1','s2','s3','s4','s5','s6'], index_col=False, dtype={'day': 'str', 'month': 'str', 'year': 'str'})
df.head()

,name,month,day,year,s1,s2,s3,s4,s5,s6
0,Mega Millions,5,26,2020,58,59,34,52,62,4
1,Mega Millions,5,29,2020,32,41,13,51,10,3
2,Mega Millions,6,2,2020,9,23,26,20,29,8
3,Mega Millions,6,5,2020,35,47,32,37,55,22
4,Mega Millions,6,9,2020,5,1,9,10,23,22


In [75]:
#The idea behind StandardScaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.
#Given the distribution of the data, each value in the dataset will have the mean value subtracted, and then divided by the standard deviation of the whole dataset (or feature in the multivariate case).
scaler = StandardScaler().fit(df[['s1','s2','s3','s4','s5','s6']].values)
transformed_dataset = scaler.transform(df[['s1','s2','s3','s4','s5','s6']].values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()

,0,1,2,3,4,5
0,1.503234,1.291146,-0.001110,1.076280,1.062441,-1.434517
1,-0.028274,0.291119,-1.167094,1.023881,-1.616848,-1.572451
2,-1.383069,-0.708908,-0.445295,-0.600495,-0.637877,-0.882780
3,0.148438,0.624461,-0.112157,0.290292,0.701768,1.048301
4,-1.618686,-1.931164,-1.389186,-1.124488,-0.947026,1.048301


In [76]:
number_of_rows= df.values.shape[0] #all rows
window_length = 2 #sliding window
number_of_features = 6 #columns count

In [77]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)

label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)
# create minibatches of 2 rows as element in train array
for i in range(0, number_of_rows-window_length):
    train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_features]

In [78]:
train.shape

(48, 2, 6)

In [79]:
label.shape

(48, 6)

In [80]:
train[0]

array([[ 1.50323351e+00,  1.29114618e+00, -1.11046079e-03,
         1.07628047e+00,  1.06244123e+00, -1.43451687e+00],
       [-2.82739845e-02,  2.91119018e-01, -1.16709429e+00,
         1.02388123e+00, -1.61684800e+00, -1.57245119e+00]])

In [81]:
train[1]

array([[-0.02827398,  0.29111902, -1.16709429,  1.02388123, -1.616848  ,
        -1.57245119],
       [-1.38306908, -0.70890814, -0.44529478, -0.60049534, -0.63787694,
        -0.88277961]])

In [82]:
label[0]

array([-1.38306908, -0.70890814, -0.44529478, -0.60049534, -0.63787694,
       -0.88277961])

In [83]:
label[1]

array([ 0.14843842,  0.62446141, -0.11215654,  0.29029181,  0.70176768,
        1.04830079])

In [84]:
batch_size = 25
model = Sequential()
# add 1st layer with 50 nodes/perceptons, each will process (2,6) matrix block
model.add(LSTM(50,      
           input_shape=(window_length, number_of_features),
           return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,           
           return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train, label, batch_size=32, epochs=8000, verbose=1)

Epoch 1/8000
48/48 [==============================] - 2s 38ms/step - loss: 0.9838 - acc: 0.1667
Epoch 2/8000
48/48 [==============================] - 0s 249us/step - loss: 0.9791 - acc: 0.1875
Epoch 3/8000
48/48 [==============================] - 0s 291us/step - loss: 0.9782 - acc: 0.1875
Epoch 4/8000
48/48 [==============================] - 0s 270us/step - loss: 0.9738 - acc: 0.1667
Epoch 5/8000
48/48 [==============================] - 0s 229us/step - loss: 0.9744 - acc: 0.2083
Epoch 6/8000
48/48 [==============================] - 0s 249us/step - loss: 0.9716 - acc: 0.1250
Epoch 7/8000
48/48 [==============================] - 0s 291us/step - loss: 0.9686 - acc: 0.1042
Epoch 8/8000
48/48 [==============================] - 0s 270us/step - loss: 0.9657 - acc: 0.1250
Epoch 9/8000
48/48 [==============================] - 0s 249us/step - loss: 0.9595 - acc: 0.1458
Epoch 10/8000
48/48 [==============================] - 0s 395us/step - loss: 0.9588 - acc: 0.1667
Epoch 11/8000
48/48 [=========

In [89]:
# adam ; 8000 epochs
to_predict=np.array([[6,7,28,59,14,7],
[8,13,68,46,35,16]])
scaled_to_predict = scaler.transform(to_predict)
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

[31 35 45 16 47 20]


In [90]:
# adam ; 8000 epochs
to_predict=np.array([[8,13,68,46,35,16],
[26,33,68,45,61,17]])
scaled_to_predict = scaler.transform(to_predict)
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

[46 37 20 23 45 15]
